In [ ]:
import gzip
import numpy as np
import pandas as pd
from time import time
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os


import cv2
src_dir='/content/drive/My Drive/covid_normal_pneumonia_test_train'
#name,feat,label=[],[],[]

feat1,label1=[],[]

feat2,label2=[],[]
#SplitTypes = []
c=0
for folders, subfolders, files in os.walk(src_dir):
    # print(folders)
    # print(subfolders)
    # print(files)
    if(len(files)!=0):
        
        for filename in files:
            c+=1
            print(c)
            file_src=os.path.join(folders,filename)
            img=cv2.imread(file_src)
            # plt.imshow(img)
            # plt.show()
            dim=(224,224)
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            # plt.imshow(resized)
            # plt.show()
            # feat1.append(resized)
            if folders.endswith('2/test'):
                # feat1.append(resized)
                feat1.append(resized)
                feat2.append(resized)
                label1.append(1)
                label2.append(1)
            elif folders.endswith('1/test'):
                # feat1.append(resized)
                feat1.append(resized)
                feat2.append(resized)
                label1.append(1)
                label2.append(0)
            elif folders.endswith('0/test'):
                feat1.append(resized)
                label1.append(0)

                
# #             plt.imshow(img)
# #             plt.show()

In [ ]:
from keras.models import load_model
 
# load model
model2 = load_model('/content/drive/My Drive/hier2.h5')

model1 = load_model('/content/drive/My Drive/hier1.h5')
model1.summary()

In [ ]:
model2.summary()

In [ ]:
X1=np.array(feat1)
y1=np.array(label1)
X2=np.array(feat2)
y2=np.array(label2)
# model1.evaluate()

model1.evaluate(X1,y1)
model2.evaluate(X2,y2)

In [ ]:
import cv2
src_dir='/content/drive/My Drive/covid_normal_pneumonia_test_train'
#name,feat,label=[],[],[]

feat2,label2=[],[]
# id2,feat2,label2=[],[],[]
# id3,feat3,label3=[],[],[]
#SplitTypes = []
c=0
for folders, subfolders, files in os.walk(src_dir):
#     print(folders.dtype)
#     print(subfolders.dtype)
#     print(files.dtype)
    if(len(files)!=0):
        
        for filename in files:
            
            # id1.append(c)
            # id3.append(c)
            file_src=os.path.join(folders,filename)
            img=cv2.imread(file_src)
            # plt.imshow(img)
            # plt.show()
            dim=(224,224)
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            
            
            if folders.endswith('2/test'):
                c+=1
                print(c)
                feat2.append(resized)
                label2.append(1)
                # feat3.append(resized)
                # label3.append(1)
                # id2.append(c)
                # id3.append(c)
                
               
            elif folders.endswith('1/test'):
                c+=1
                print(c)
                feat2.append(resized)
                label2.append(0)
                # id2.append(c)
                # feat3.append(resized)
                # label3.append(0)
                # id3.append(c)
            
            # elif folders.endswith('0/test'):
            #     c+=1
            #     print(c)
            #     feat2.append(resized)
            #     label2.append(0)
            #     id2.append(c)


from keras.applications import ResNet50, VGG19
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model

vgg=VGG19(include_top=False, weights="imagenet",input_tensor=Input(shape=(224,224,3)))
for layer in vgg.layers:
  layer.trainable = False
# x = Flatten()(resnet.output)
vgg.summary()
# resnet.input

feat_proc=[]
len(feat2)

for i in range(len(feat2)):
    # imgx = image.load_img(f, target_size=(224, 224))
    # x = image.img_to_array(imgx)
    x=feat2[i]
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feat_proc.append(x)
    
fx=[]

for f in tqdm(feat_proc):
    features = vgg.predict(f)
    features = features.flatten()
    fx.append(features)
    
fx

In [ ]:
X=np.array(fx)
num=len(X[0])

colx=[]
for i in range(num):
    colx.append('x'+str(i))
    
df=pd.DataFrame(data=X,columns=colx)
df['Label']=label2
df.to_csv('/content/drive/My Drive/level1 test_v3.csv')
# !cp data.csv "drive/My Drive/"
